# Total energy convergence
[Link to tutorial](https://octopus-code.org/documentation/13/tutorial/basics/total_energy_convergence/)

In this tutorial we will show how to converge a quantity of interest (the total energy) with respect to the parameters that define the real-space grid. For this we will use two examples, the Nitrogen atom from the [Basic input options](2-Basic_input_options.ipynb) tutorial, and a methane molecule.


In [ ]:
import os
import subprocess
from string import Template

import pandas as pd

from postopus import Run

In [ ]:
!mkdir -p 3-total_energy_convergence

In [ ]:
cd 3-total_energy_convergence

## Nitrogen atom: finding a good spacing

The key parameter of a real-space calculation is the spacing between the points of the mesh. The default option in **Octopus** is to use a Cartesian grid. This is a regular grid where the spacing along each Cartesian direction is the same. The first step in any calculation should then be making sure that this spacing is good enough for our purposes. This should be done through a convergence study, very similar to the ones performed in plane-wave calculations.

The needed spacing essentially depends on the pseudopotentials that are being used. The idea is to repeat a series of ground-state calculations, with identical input files except for the grid spacing. There are many different ways of doing it, the simplest one being to change the input file by hand and run **Octopus** each time. Another option is to use a little python script like the one bellow.

As a first example, we will use the Nitrogen atom from the previous tutorial.


In [ ]:
inp_template_Nitrogen = Template(
    """
CalculationMode = gs
UnitsOutput = eV_Angstrom

stdout = 'stdout_td_$Spacing_value.txt'
stderr = 'stderr_td_$Spacing_value.txt'

Nitrogen_mass = 14.0

%Species
'N' | species_pseudo | set | standard | lmax | 1 | lloc | 0 | mass | Nitrogen_mass
%

XYZCoordinates = 'N.xyz'

ExtraStates = 1
%Occupations
2 | 1 | 1 | 1
%

BoxShape = sphere
Radius = 5.0*angstrom
Spacing = $Spacing_value*angstrom
"""
)

In [ ]:
N_xyz_string = """


1
This is a comment line
N 0 0 0
"""

In [ ]:
def get_scf_info(run_path: str):
    """
    Extract Total energy , seigen, peigen from info.
    """
    info = Run(run_path).default.scf.info()

    total_energy = [lines for lines in info if "Total       = " in lines][-1]
    total_energy = float(total_energy.split("=")[-1])

    s_eigen = [lines for lines in info if "1   --" in lines][-1]
    s_eigen = float(s_eigen.split()[2])

    p_eigen = [lines for lines in info if "2   --" in lines][-1]
    p_eigen = float(p_eigen.split()[2])
    return total_energy, s_eigen, p_eigen

### Create runs for different spacings
What this script does is quite simple: it runs **Octopus** for a list of spacings (0.26, 0.24, ..., 0.14) and after each calculation it collects the values of the total energy and of the eigenvalues. In order to change the value of the spacing between each calculation, it uses the [python string template](https://docs.python.org/3/library/string.html#string.Template). Also for each spacing a separate directory is used.


In [ ]:
list_of_spacing = [0.28, 0.26, 0.24, 0.22, 0.20, 0.18, 0.16, 0.14]

for spacing in list_of_spacing:
    print(f"Running octopus for spacing: {spacing}")
    os.makedirs(f"Nitrogen/run_{spacing}")
    with open(f"Nitrogen/run_{spacing}/inp", "w") as inp_file:
        inp_file.write(inp_template_Nitrogen.substitute(Spacing_value=spacing))
    with open(f"Nitrogen/run_{spacing}/N.xyz", "w") as N_xyz_file:
        N_xyz_file.write(N_xyz_string)
    subprocess.run("octopus", shell=True, cwd=f"Nitrogen/run_{spacing}")

### Retrieve and combine data for different spacings

In [ ]:
table = []
for spacing in list_of_spacing:
    total_energy, s_eigen, p_eigen = get_scf_info(f"Nitrogen/run_{spacing}")
    table.append([spacing, total_energy, s_eigen, p_eigen])
df = pd.DataFrame(table, columns=["Spacing", "Total_Energy", "S_eigen", "P_eigen"])
df

In [ ]:
# shift values relativ to the last run (the one with the lowest spacing)
df[df.columns[1:]] = df[df.columns[1:]] - df[df.columns[1:]].iloc[-1]
df

In [ ]:
# plot total energy, S-eigen, P-eigen trend against Spacing
df.plot(
    x="Spacing",
    y=["Total_Energy", "S_eigen", "P_eigen"],
    style="o-",
    grid=True,
    ylabel="Error (eV)",
    xlabel="Spacing (Å)",
)

The results, for this particular example, are shown in the figure.
In this figure we are actually plotting the error with respect to the most accurate results (smallest spacing).
That means that we are plotting the '''difference''' between the values for a given spacing and the values for a spacing of 0.14 Å.
So, in reading it, note that the most accurate results are at the left (smallest spacing).
A rather good spacing for this nitrogen pseudopotential seems to be 0.18 Å.
However, as we are usually not interested in total energies, but in energy differences, probably a larger one may also be used without compromising the results.


## Methane molecule
We will now move on to a slightly more complex system, the methane molecule CH<sub>4</sub>, and add a convergence study with respect to the box size.

### Input
As usual, the first thing to do is create an input file for this system. From our basic chemistry class we know that methane has a tetrahedral structure. The only other thing required to define the geometry is the bond length between the carbon and the hydrogen atoms. If we put the carbon atom at the origin, the hydrogen atoms have the coordinates given in the following input file:


In [ ]:
inp_template_methane = Template(
    """
stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs
UnitsOutput = eV_Angstrom

Radius = $Radius_value*angstrom
Spacing = $Spacing_value*angstrom

CH = 1.2*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%
"""
)

Here we define a variable CH that represents the bond length between the carbon and the hydrogen atoms, which simplifies the writing of the coordinates. We start with a bond length of 1.2 Å but that's not so important at the moment, since we can optimize it later. (You should not use 5 Å or so, but something slightly bigger than 1 Å is fine.)

Some notes concerning the input file:
* We do not tell **Octopus** explicitly which [BoxShape](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/boxshape) to use. The default is a union of spheres centered around each atom. This turns out to be the most economical choice in almost all cases.
* We also do not specify the %[Species](https://www.octopus-code.org/documentation//13/variables/system/species/species) block. In this way, **Octopus** will use default pseudopotentials for both Carbon and Hydrogen. This should be OK in many cases, but, as a rule of thumb, you should do careful testing before using any pseudopotential for serious calculations.


In [ ]:
os.makedirs("3-methane_example/variable_spacing/run_0.22")
with open("3-methane_example/variable_spacing/run_0.22/inp", "w") as inp_file:
    inp_file.write(
        inp_template_methane.substitute(Spacing_value=0.22, Radius_value=3.5)
    )

subprocess.run("octopus", shell=True, cwd="3-methane_example/variable_spacing/run_0.22")

In [ ]:
!cat 3-methane_example/variable_spacing/run_0.22/static/info

Now the question is whether these values are converged or not. This will depend on two things: the [BoxShape](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/boxshape), as seen above, and the [Species](https://www.octopus-code.org/documentation//13/variables/system/species/species). Just like for the Nitrogen atom, the only way to answer this question is to try other values for these variables.


In [ ]:
!grep -am 1 -A 26 "not fully" 3-methane_example/variable_spacing/run_0.22/static/info

###  Convergence with the spacing


As before, we will keep all entries in the input file fixed except for the spacing that we will make smaller by 0.02 Å all the way down to 0.1 Å. So you have to run **Octopus** several times. We use a similar script as the one from the Nitrogen atom example:


In [ ]:
spacing_list = [0.20, 0.18, 0.16, 0.14, 0.12, 0.10]

for spacing in spacing_list:
    print(f"Running octopus for spacing: {spacing}")
    os.makedirs(f"3-methane_example/variable_spacing/run_{spacing}")
    with open(f"3-methane_example/variable_spacing/run_{spacing}/inp", "w") as inp_file:
        inp_file.write(
            inp_template_methane.substitute(Spacing_value=spacing, Radius_value=3.5)
        )
    subprocess.run(
        "octopus", shell=True, cwd=f"3-methane_example/variable_spacing/run_{spacing}"
    )

In [ ]:
table_methane = []
for spacing in spacing_list:
    run_path = f"3-methane_example/variable_spacing/run_{spacing}"
    total_energy, s_eigen, p_eigen = get_scf_info(run_path)
    table_methane.append([spacing, total_energy, s_eigen, p_eigen])
methane_df = pd.DataFrame(
    table_methane, columns=["Spacing", "Total_Energy", "S_eigen", "P_eigen"]
)
methane_df

In [ ]:
methane_df.plot(
    x="Spacing",
    y=["Total_Energy"],
    style="o-",
    grid=True,
    ylabel="Total Energy (eV)",
    xlabel="Spacing (Å)",
);

As you can see from this picture, the total energy is converged to within 0.1 eV for a spacing of 0.18 Å. So we will use this spacing for the next calculations.

### Convergence with the radius


Now we will see how the total energy changes with the [Radius](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/radius) of the box. We will change the radius in steps of 0.5 Å. You can change the input file by hand and run Octopus each time, or again use a small script:


In [ ]:
radius_list = [2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

for radius in radius_list:
    print(f"Running octopus for radius: {radius}")
    os.makedirs(f"3-methane_example/variable_radius/run_{radius}")
    with open(f"3-methane_example/variable_radius/run_{radius}/inp", "w") as inp_file:
        inp_file.write(
            inp_template_methane.substitute(Spacing_value=0.18, Radius_value=radius)
        )
    subprocess.run(
        "octopus", shell=True, cwd=f"3-methane_example/variable_radius/run_{radius}"
    )

In [ ]:
table_methane_radius = []
current_folder = os.getcwd()
for radius in radius_list:
    run_path = f"3-methane_example/variable_radius/run_{radius}"
    total_energy, s_eigen, p_eigen = get_scf_info(run_path)
    table_methane_radius.append([radius, total_energy, s_eigen, p_eigen])
methane_radius_df = pd.DataFrame(
    table_methane_radius, columns=["Radius", "Total_Energy", "S_eigen", "P_eigen"]
)
methane_radius_df

In [ ]:
methane_radius_df.plot(
    x="Radius",
    y=["Total_Energy"],
    style="o-",
    grid=True,
    ylabel="Total Energy (eV)",
    xlabel="Radius (Å)",
)

If we again ask for a convergence up to 0.1 eV we should use a radius of 3.5 Å. How does this compare to the size of the molecule? Can you explain why the energy increases (becomes less negative) when one decreases the size of the box?


[Go to *4-Visualisation.ipynb*](4-Visualisation.ipynb)